In [ ]:
print("""MIT License

Copyright (c) 2022 Mauricio Carvalho Mathias de Paulo and Danilo 

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.""")

## Libraries

In [ ]:
from datetime import datetime
import pandas as pd
import urllib
import os
from zipfile import ZipFile
import subprocess
import numpy as np
import matplotlib.pyplot as plt
import re
import io
import urllib.request
import scipy
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

## Downloading data

In [ ]:
start_doy = 1
end_doy = 365
end_year=2019
start_year=2013
#year=2019
output_folder = "MyDrive/0braz"
station="braz"
#We recommend running the code one year at a time, because the servers might disable long download sequences.

In [ ]:
#from google.colab import drive
mount_folder = "/mnt/ext4/mauricio/artigos/braz_iono_series/"
#drive.mount(mount_folder)
folder = os.path.join(mount_folder,output_folder)
os.makedirs(folder,exist_ok=True)
os.chdir(folder)

In [ ]:
print(folder)

# Baixar os programas de conversão e limpeza do rinex

In [ ]:
if not os.path.exists("crx2rnx"):
  !wget https://terras.gsi.go.jp/ja/crx2rnx/RNXCMP_4.1.0_src.tar.gz
  !tar -xzvf RNXCMP_4.1.0_src.tar.gz
  !gcc ./RNXCMP_4.1.0_src/source/crx2rnx.c -o crx2rnx
if not os.path.exists("teqc"):
  !wget https://www.unavco.org/software/data-processing/teqc/development/teqc_CentOSLx86_64s.zip -O teqc.zip
  !unzip -o teqc.zip 

# Baixar os arquivos rinex e executar a limpeza dos arquivos

In [ ]:
for year in range (start_year,end_year+1):
  for doy in range (start_doy,end_doy+1+(0 if year%4!=0 else 1)):
    link="""https://geoftp.ibge.gov.br/informacoes_sobre_posicionamento_geodesico/rbmc/dados/{2}/{0:03}/{1}{0:03}1.zip""".format(doy,station,year)
    rbmcfile=link.split("/")[-1]
    print(link)
    zipFile=str(year)+rbmcfile
    if not os.path.exists(zipFile) or os.path.getsize(zipFile)<1024:
      !wget "$link" -O $zipFile
      !chmod 777 crx2rnx teqc 
      !unzip -o $zipFile 
      crxfile=rbmcfile.replace(".zip",'.{}d'.format(year%100))
      if os.path.exists(crxfile):
          !./crx2rnx -f $crxfile
      #print("Output file should be {}. Please check the Files tab.".format("rjni2361.21d"[:-1]+"o"))
      rnx2file=crxfile[:-1]+"o"
      newFile=rnx2file.replace(".","gps.")
      !./teqc -E -C -R -S -O.obs L1L2C1P2S1S2 +out $newFile $rnx2file 
      print("RINEX 2 file saved to {}".format(newFile))
      #Let's fix the lack of wavelength factor
      !sed '/^.*APPROX POSITION XYZ.*/a \     1     1                                                WAVELENGTH FACT L1\/2' -i $newFile
    else:
      print("File exists {}".format(zipFile))

# Criação da função para criar os arquivos de configuração do rinex_ho

In [ ]:
#helper function to get month from doy and year
def getMonth(year,doy):
  weird_date=np.datetime64(year, 'Y')+np.timedelta64(doy,'D')
  return  int(weird_date.astype('datetime64[M]').astype(int)%12+1)

In [ ]:
def make_rinex_ha(doy, year, station, path):
  month=getMonth(year,doy)
  rinex_ha = """{3}/{0}{1:03}1gps.{2}o             			//obs file
{3}/{0}{1:03}1.{2}n             			//nav file
{3}/{0}{1:03}1c.{2}o             			//new obs file
{3}/{0}{1:03}_{2}_Out              			//Name of output files
4115014.0848 -4550641.5491 -1741444.0190    	//Receiver coordinates (if 0.0 => Try to read coordinates from RINEX)
10                                            	//Elevation Mask (means that observables under this mask won't be corrected)
1	                 			//Save output and log files (yes=1; no=0)
2		         			//0 = Tec from raw pseudorange; 1 = TEC from smothed pseudorange by phase; 2 = TEC from GIM;3 = TEC from carrier levelled by code
{3}/ionex/codg{1:03}0.{2}i             			//Name of the CODE map: In the case before option "To use CODE map" = 1 (ftp://ftp.unibe.ch/aiub/CODE/)  
0//-10.708                    			//DCB (P1-P2) = receiver bias in ns (nano-segundos): In the case computing TEC from pseudorange 
{3}/dcb/P1C1{2}{4:02}.DCB          			        //DCB (P1-C1) - P1C1 bias for satellite - Found at: ftp://ftp.unibe.ch/aiub/CODE/
{3}/dcb/P1P2{2}{4:02}.DCB          			        //DCB (P1-P2) - P1P2 bias for satellites - Found at: ftp://ftp.unibe.ch/aiub/CODE/
0                        			//0 = Dipolar model; 1 = CGM from PIM; 2 = IGRF model
IGRF_COEF/IGRF13.COF           			//Name of the IGRF coefficients""".format(station,doy,year%100,path,month)
  #print(rinex_ha)
  f = open("rinex_ha.inp", "w")
  f.write(rinex_ha)
  f.close()


# Baixando mapas de ionosfera

In [ ]:
def downloadIonex(doy,year):
  baseurl=f"ftp://igs.ign.fr/pub/igs/products/ionosphere/{year}/{doy:03}/codg{doy:03}0.{year%100}i.Z"
  os.makedirs("ionex",exist_ok=True)
  local_filename = os.path.join("ionex",baseurl.split('/')[-1])
  dcb_file=local_filename.replace('.Z','')
  if not os.path.exists(dcb_file):
    print("Downloading ", baseurl, dcb_file)
    #urllib.request.urlretrieve(baseurl, local_filename)
    !wget $baseurl -O $local_filename
    print("Saved ",local_filename)
    if os.path.getsize(local_filename) == 0:
      print("Trying to get rapid, because final ionex was not found.")
      baseurl=baseurl.replace("codg","corg")
      !wget $baseurl -O $local_filename
    !gunzip $local_filename -f

In [ ]:
for year in range (start_year,end_year+1):
  for doy in range (start_doy,end_doy+1):
    downloadIonex(doy,year)

# Baixando os arquivos de atraso de hardware

In [ ]:
dcbfolder=os.path.join(folder,"dcb")
os.makedirs(dcbfolder,exist_ok=True)
for year in range (start_year,end_year+1):
  for month in range (1,13):
    p1c1=f"P1C1{year%100}{month:02}.DCB"
    p1p2=f"P1P2{year%100}{month:02}.DCB"
    if not os.path.exists(f"{dcbfolder}/{p1c1}"):
      !wget "http://ftp.aiub.unibe.ch/CODE/{year}/{p1c1}.Z" -O "{dcbfolder}/{p1c1}.Z"
      !uncompress -f "{dcbfolder}/{p1c1}.Z" 
    if not os.path.exists(f"{dcbfolder}/{p1p2}"):
      !wget "http://ftp.aiub.unibe.ch/CODE/{year}/{p1p2}.Z" -O "{dcbfolder}/{p1p2}.Z"
      !uncompress -f "{dcbfolder}/{p1p2}.Z" 
!ls {dcbfolder}

# Compilando o Rinex_ho

In [ ]:
!rm -rf /content/drive/MyDrive/0rjni/rinex_ho-main

In [ ]:
!cd ~


In [ ]:
%cd $folder
if os.path.exists("rinex_ho"):
  %cd rinex_ho
else:
  !git clone "https://github.com/mauriciodev/rinex_ho.git"
  !git checkout VTEC
  %cd rinex_ho
  !cmake .
  !make

# Executando o Rinex_ho para cada conjunto de arquivos rinex

In [ ]:
os.chdir(os.path.join(folder,"rinex_ho"))
overwrite=False
!chmod 777 Rinex_ho
for year in range (start_year,end_year+1):
  for doy in range (start_doy,end_doy+1+(0 if year%4!=0 else 1)):
    make_rinex_ha(doy,year,station,folder)
    tecFile="{3}/{0}{1:03}_{2}_Out_tec.txt".format(station,doy,year%100,folder)
    #less than 1kb is probably a faulty file
    if (not os.path.exists(tecFile)) or (overwrite==True) or (os.path.getsize(tecFile) < 1024):
      print("Running Rinex_ho for doy {0}, year {1}, station {2}, folder: {3}".format(doy,year,station,folder))
      subprocess.run("./Rinex_ho", shell=True, stdout=subprocess.DEVNULL)
    else:
      print("TEC file {0} already exists. Skipping.".format(tecFile))

# Visualização dos resultados da 2ª ordem

Fazer o laço para todos os doys e para os outros arquivos _Out

In [ ]:
os.chdir(folder)

In [ ]:
#This might take a while
def makeTimeSeries(sufix, station):
  results=[]
  for year in range (start_year,end_year+1): 
    for doy in range(start_doy,end_doy+1+(0 if year%4!=0 else 1)):
      df=pd.read_fwf("{1}{0:03}_{2}_{3}.txt".format(doy,station,year%100,sufix))
      df.replace(to_replace=9999999,value=np.NaN,inplace=True)
      df.replace(to_replace=999999,value=np.NaN,inplace=True)
      df['h:m:s'] = np.NaN
      media = df.mean(skipna=True).mean()
      media = (media if not np.isnan(media) else 0)
      results.append(media)
  return results
  #print(results_I2L1)
  #print(doy)
  #df
results_I2L1=makeTimeSeries("Out_I2L1",station)
print("I2L1 finalizado.")
results_I2L2=makeTimeSeries("Out_I2L2",station)
print("I2L2 finalizado.")
results_I3L1=makeTimeSeries("Out_I3L1",station)
print("I3L1 finalizado.")
results_I3L2=makeTimeSeries("Out_I3L2",station)
print("I3L2 finalizado.")
results_tec=makeTimeSeries("Out_tec",station)
print("TEC finalizado.")

#Replicar para os demais

#Para salvar a série temporal 

In [ ]:
os.makedirs("results",exist_ok=True)
np.savetxt('results/results_I2L1.txt',np.array(results_I2L1))
np.savetxt('results/results_I2L2.txt',np.array(results_I2L2))
np.savetxt('results/results_I3L1.txt',np.array(results_I3L1))
np.savetxt('results/results_I3L2.txt',np.array(results_I3L2))
np.savetxt('results/results_tec.txt',np.array(results_tec))

In [ ]:
results_I2L1 = np.loadtxt('results/results_I2L1.txt')
results_I2L2 = np.loadtxt('results/results_I2L2.txt')
results_I3L1 = np.loadtxt('results/results_I3L1.txt')
results_I3L2 = np.loadtxt('results/results_I3L2.txt')
results_tec = np.loadtxt('results/results_tec.txt')

Plotar um gráfico para cada arquivo result e um gráfico com os quatro

In [ ]:
plt.plot(results_I2L1)
plt.title('I2L1')
plt.xlabel('Dia do ano')
plt.ylabel('atraso ionosférico de 2ª ordem')
plt.show()

In [ ]:
x_axis=np.arange(start_doy,end_doy+1)
plt.plot(results_I2L1)
#plt.xticks(np.arange(start_doy,end_doy+1, 0.5)) #intervalos do eixo x
plt.title('I2L1')
plt.xlabel('Dia do ano')
plt.ylabel('Atraso ionosférico de 2ª ordem na L1')
plt.savefig('results/I2L1.png') #antes do show
plt.show()

In [ ]:
plt.plot(results_I2L2)
plt.title('I2L2')
plt.xlabel('Dia do ano')
plt.ylabel('Atraso ionosférico de 2ª ordem na L2')
plt.show()

In [ ]:
plt.plot(results_I3L1)
plt.title('I3L1')
plt.xlabel('Dia do ano')
plt.ylabel('Atraso ionosférico de 3ª ordem na L1')
plt.show()

In [ ]:
plt.plot(results_I3L2)
plt.title('I3L2')
plt.xlabel('Dia do ano')
plt.ylabel('Atraso ionosférico de 3ª ordem na L2')
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(results_I3L2, label="L2 - 3a ordem")
plt.plot(results_I3L1, label="L1 - 3a ordem")
plt.plot(results_I2L2, label="L2 - 2a ordem")
plt.plot(results_I2L1, label="L1 - 2a ordem")
plt.title('Efeitos ionosféricos de ordem superior')
plt.xlabel('Dia do ano')
plt.ylabel('Efeito ionosférico (m)')
plt.legend()
plt.show()

In [ ]:
plt.plot(results_tec)
plt.title('TEC ao longo do ano')
plt.xlabel('Dia do ano')
plt.ylabel('TEC')
plt.show()

# Space weather

In [ ]:
for year in range (start_year,end_year+1):
  baseurl = f"ftp://ftp.gfz-potsdam.de/pub/home/obs/Kp_ap_Ap_SN_F107/Kp_ap_Ap_SN_F107_{year}.txt"
  os.makedirs("f107",exist_ok=True)
  local_filename = os.path.join("f107",baseurl.split('/')[-1])
  f107_file=local_filename.replace('.gz','')
  print("Downloading ", baseurl)
  !wget $baseurl -O $local_filename
  print("Saved ",local_filename)

In [ ]:
f107=[]
for year in range (start_year,end_year+1):
  baseurl = f"ftp://ftp.gfz-potsdam.de/pub/home/obs/Kp_ap_Ap_SN_F107/Kp_ap_Ap_SN_F107_{year}.txt"
  os.makedirs("f107",exist_ok=True)
  local_filename = os.path.join("f107",baseurl.split('/')[-1])
  f107_file=local_filename.replace('.gz','')
  df=pd.read_csv(f107_file,comment='#',header=None, delim_whitespace=True)
  fcolumn=df.columns[-2]
  df.loc[df[fcolumn] <= 0, fcolumn] = None
  f107_series=df[fcolumn].replace(-1,None).ffill()
  f107.append(f107_series.to_numpy())
  #print(df[fcolumn].to_numpy())
  #print("Downloading ", baseurl)
  #print("Saved ",local_filename)
f107_concat=np.concatenate(f107)
np.savetxt('results/f107.txt',np.array(f107_concat))


In [ ]:
# Exporting the table

In [ ]:
from datetime import datetime
firstDate=datetime(year=2013, month=1,day=1)
datelist = pd.date_range(firstDate, periods=len(results_I3L2)).tolist()
d={"date": datelist, "I2L1": np.array(results_I2L1),"I2L2": np.array(results_I2L2),"I3L1": np.array(results_I3L1), "I3L2": np.array(results_I3L2), "VTEC": np.array(results_tec), "f107cm": f107_concat}
outdf=pd.DataFrame(data=d)
outdf.to_csv("iono_series.csv")

In [ ]:
results_f107 = np.loadtxt('results/f107.txt')

In [ ]:
plt.plot(results_f107, label='F10.7cm (fluxo solar)')
plt.xlabel('Dia do ano')
plt.ylabel('F10.7cm')
plt.legend()

In [ ]:
results_f107
ind=np.argpartition(results_f107, -5)[-5:]
results_f107[ind]

In [ ]:
plt.figure(figsize=(20,20))
plt.plot(results_I2L2, label="L2 - 2a ordem")
plt.plot(results_f107/100000.*5, label="F10.7cm (deslocado)")
plt.title('Atrasos ionosféricos de 2ª ordem')
plt.xlabel('Dia do ano')
plt.ylabel('Atraso ionosférico de 2ª ordem (m)')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(20,20))
plt.plot(results_I3L2, label="L2 - 3a ordem")
plt.plot(results_I3L1, label="L1 - 3a ordem")
plt.plot(results_I2L2, label="L2 - 2a ordem")
plt.plot(results_I2L1, label="L1 - 2a ordem")
plt.plot(results_f107/100000., label="F10.7cm")
plt.title('Atrasos ionosféricos de 2ª ordem')
plt.xlabel('Dia do ano')
plt.ylabel('Atraso ionosférico de 2ª ordem (m)')
plt.legend()
plt.show()

In [ ]:
outdf.max()

# Correlação

In [ ]:
results_f107.shape
results_I3L2.shape

In [ ]:
import scipy.stats
print(scipy.stats.pearsonr(results_f107,results_I3L2))
print(scipy.stats.pearsonr(results_f107,results_I3L1))
print(scipy.stats.pearsonr(results_f107,results_I2L2))
print(scipy.stats.pearsonr(results_f107,results_I2L1))
print(scipy.stats.pearsonr(results_f107,results_tec))

In [ ]:
compareSeries=results_I2L1
plt.scatter(results_f107,compareSeries)
plt.annotate("r-squared = {:.3f}".format(r2_score(results_f107, compareSeries)), (0, 1), xycoords='figure points')
y_test, y_predicted = results_f107.reshape(-1,1), np.array(compareSeries).reshape(-1,1)
plt.plot(y_test, LinearRegression().fit(y_test, y_predicted).predict(y_test), color="red")

In [ ]:
outdf.describe()